In [1]:
import sqlite3
# import ibis
import sys
import pandas as pd
sys.path.append("..")
from rcn_py import database
from rcn_py import neo4j_scopus
from neo4j import GraphDatabase
import os

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## SQLite DB
    author-paper table
    author info table
    publication table

In [12]:
con = sqlite3.connect("/Users/jennifer/RCSNA/data/tutorial.db")
cur = con.cursor()

In [12]:
cur.execute("CREATE TABLE test_paper(doi primary key, title, abstract)")

In [13]:
cur.execute("CREATE TABLE test_authors(orcid primary key, name, country)")


In [16]:
cur.execute("CREATE TABLE test_author_papers2(orcid, doi, primary key(orcid, doi) FOREIGN KEY(orcid) REFERENCES test_authors(orcid), FOREIGN KEY(doi) REFERENCES test_paper(doi))")

In [55]:
con.commit()

In [56]:
con.close()

## Insert data

In [72]:
database.insert_database("/Users/jennifer/RCSNA/data/tutorial.db", '',"Peter Kalverla")

'Done.'

In [ ]:
database.insert_coauthors_pub("/Users/jennifer/RCSNA/data/tutorial.db", "Peter Kalverla")

In [ ]:
database.insert_cocoauthors("/Users/jennifer/RCSNA/data/tutorial.db")

### Fetch data from database

In [ ]:
con = sqlite3.connect("/Users/jennifer/RCSNA/data/tutorial.db")
cur = con.cursor()

# Topic modeling (title or abstract) to cluster papers
clusters, idx2topics = database.pub_cluster(cur)

In [ ]:
# get coauthor relationship
links = database.fetch_relationships(cur)

In [ ]:
# assign author's group as the most frequent paper group
orcid_group = database.author_cluster(cur, clusters)

cur.close()

In [ ]:
# pyvis
database.build_network_database("Database")

### Store more data

In [4]:
con = sqlite3.connect("/Users/jennifer/RCSNA/data/tutorial.db")
cur = con.cursor()

In [6]:
con_rsd = sqlite3.connect("/Users/jennifer/RCSNA/data/rsd.db")
cur_rsd = con_rsd.cursor()

In [18]:
res_rsd = cur_rsd.execute("""
SELECT orcid, given_name, family_name, affiliation 
FROM author 
WHERE affiliation = 'Netherlands eScience Center'
GROUP BY orcid
UNION
SELECT orcid, given_name, family_name, affiliation 
FROM contributor 
WHERE affiliation = 'Netherlands eScience Center'
GROUP BY orcid
""")
escience_member = res_rsd.fetchall()

In [25]:
escience_member[0]


('0000-0001-5301-3276',
 'Jesus',
 'Garcia Gonzalez',
 'Netherlands eScience Center')

In [2]:
def insert_escience_rsd_member(db_path):
    con_rsd = sqlite3.connect("/Users/jennifer/RCSNA/data/rsd.db")
    cur_rsd = con_rsd.cursor()
    res_rsd = cur_rsd.execute("""
        SELECT orcid, given_name, family_name, affiliation 
        FROM author 
        WHERE affiliation = 'Netherlands eScience Center'
        GROUP BY orcid
        UNION
        SELECT orcid, given_name, family_name, affiliation 
        FROM contributor 
        WHERE affiliation = 'Netherlands eScience Center'
        GROUP BY orcid
        """)
    escience_members = res_rsd.fetchall()
    for member in escience_members:
        orcid_id = member[0]
        fullname = member[1] + " " + member[2]
        database.insert_database(db_path, orcid_id, fullname)

    con_rsd.commit()
    con_rsd.close()
    return "Insertion complete"
    

In [3]:
insert_escience_rsd_member("/Users/jennifer/RCSNA/data/tutorial.db")

'Insertion complete'

In [14]:
# cur.execute("DELETE FROM author_publication WHERE doi = 'None'")
con.commit()
con.close()

## Neo4j

In [ ]:
# remote AuraDB, only one for free'
# uri = "neo4j+s://a93e1a64.databases.neo4j.io"
# user = "neo4j"
# password = "ifVwf-BGTZt1ltrNgVbh5Q6aGSxQSksQJSxBUbJb1BE"

In [2]:
# local AuraDB
uri = "bolt://localhost:7687"
user = "neo4j"
password = "zhiningbai"

In [3]:
check_verify =  GraphDatabase.driver(uri, auth=(user, password))
check_verify.verify_connectivity()

/var/folders/fp/vghv4nhj5zg720lgbcqs7x300000gn/T/ipykernel_24418/2331392860.py:2: ExperimentalWarning: The configuration may change in the future.
  check_verify.verify_connectivity()


'Neo4j/5.6.0'

In [ ]:
# Add constraint Cypher
"""
CREATE CONSTRAINT pub_doi IF NOT EXISTS
FOR (p:Publication) REQUIRE p.doi IS UNIQUE

CREATE CONSTRAINT scopus_id IF NOT EXISTS
FOR (n:Person) REQUIRE n.scopus_id IS UNIQUE
"""

In [5]:
driver = GraphDatabase.driver(uri, auth=(user, password))
session = driver.session(database="neo4j")
session.execute_write(neo4j_scopus.add_constraint) 

In [6]:
file_subject = ["Medicine", "Biochemistry, Genetics and Molecular Biology", "Social Sciences",
                "Engineering", "Physics and Astronomy", "Computer Science", "Environmental Science",
                "Agricultural and Biological Sciences", "Earth and Planetary Sciences",
                "Chemistry", "Psychology", "Neuroscience", "Mathematics",
                "Immunology and Microbiology", "Materials Science", "Multidisciplinary",
                "Arts and Humanities", "Chemical Engineering", "Pharmacology, Toxicology and Pharmaceutics",
                "Business, Management and Accounting", "Energy", "Nursing",
                "Economics, Econometrics and Finance", "Health Professions",
                "Decision Sciences", "Veterinary", "Dentistry"
                ]

In [85]:
len(file_subject)

27

In [10]:
# Execution
# 13, 

with GraphDatabase.driver(uri, auth=(user, password)) as driver:
    driver.verify_connectivity()
    with driver.session(database="neo4j") as session:
        # Create nodes & edges
        # Traversing csv files
        for subject in file_subject:
            for n in range(1, 13):
                filename = subject + str(n)
                path = "/Users/jennifer/scopus_data/year2014/"+filename+".csv"
                if os.path.exists(path):
                    # Skipping bad lines (very rare occurrence): + ", on_bad_lines = 'skip'"
                    df = pd.read_csv(path)
            
                    session.execute_write(neo4j_scopus.neo4j_create_people, df, subject) 
                    session.execute_write(neo4j_scopus.neo4j_create_publication, df, subject)
                    session.execute_write(neo4j_scopus.neo4j_create_author_pub_edge, df)
                    print (subject+str(n)+" is Done!")  
                else:
                    if n == 1:
                        print("Cannot find file: "+filename)
                    break    
            

/var/folders/fp/vghv4nhj5zg720lgbcqs7x300000gn/T/ipykernel_24418/3125506189.py:5: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


Medicine1 is Done!
Medicine2 is Done!
Medicine3 is Done!
Medicine4 is Done!
Medicine5 is Done!
Biochemistry, Genetics and Molecular Biology1 is Done!
Biochemistry, Genetics and Molecular Biology2 is Done!
Biochemistry, Genetics and Molecular Biology3 is Done!
Social Sciences1 is Done!
Engineering1 is Done!
Physics and Astronomy1 is Done!
Physics and Astronomy2 is Done!
Computer Science1 is Done!
Environmental Science1 is Done!
Agricultural and Biological Sciences1 is Done!
Agricultural and Biological Sciences2 is Done!
Earth and Planetary Sciences1 is Done!
Chemistry1 is Done!
Psychology1 is Done!
Neuroscience1 is Done!
Mathematics1 is Done!
Immunology and Microbiology1 is Done!
Materials Science1 is Done!
Multidisciplinary1 is Done!
Arts and Humanities1 is Done!
Chemical Engineering1 is Done!
Pharmacology, Toxicology and Pharmaceutics1 is Done!
Business, Management and Accounting1 is Done!
Energy1 is Done!
Nursing1 is Done!
Economics, Econometrics and Finance1 is Done!
Health Professi

In [42]:
session.close()
driver.close()